# PRACTICAL TASK 3

In [257]:
import pandas as pd
import apyori as apriori
import numpy as np
import collections
import itertools
from anytree import Node, RenderTree, search

## TASK 1

In [258]:
itemset = ["ABCD", "ACDF", "ACDEG", "ABDF", "BCG", "DFG", "ABG", "CDFG"]
itemset_list = [list(str) for str in itemset]
tid = [n for n in range(1, 9)]

In [259]:
df = pd.DataFrame(tid, columns=["tid"])

unique_items = list(collections.OrderedDict.fromkeys("".join(itemset)).keys())

for i in range(len(unique_items)):
    df[unique_items[i]] = [1 if unique_items[i] in s else 0 for s in itemset_list]

df.head()

,tid,A,B,C,D,F,E,G
0,1,1,1,1,1,0,0,0
1,2,1,0,1,1,1,0,0
2,3,1,0,1,1,0,1,1
3,4,1,1,0,1,1,0,0
4,5,0,1,1,0,0,0,1


In [260]:
support = 3/8

In [261]:
F = []

In [262]:
# первый этап
print(f"Step 0")

F.append(dict())

# подсчет поддержки для всех уникальных элементов
for item in unique_items:
    local_support = np.sum(df[item]) / df.shape[1]
    print(f"support of {item}: \t{local_support}")
    if local_support >= support:
        F[0][item] = np.sum(df[item]) / df.shape[1]

print(f"frequent sets: {F[0]}")
print()

# следующие шаги
k = 1

while bool(F[k-1]): # пока есть часто встречающиеся наборы
    print(f"Step {k}")
    C = dict()
    combinations = list(itertools.combinations(unique_items, k+1)) #получаем все возможные комбинации элементов

    print(f"amount of possible combinations of {k+1} elements: \t{len(combinations)}")

    # генерация комбинаций с учетом поддержки наборов меньшего размера
    for i in reversed(range(len(combinations))):
        if k > 1:
            local_combinations = list(itertools.combinations(combinations[i], k))
        else:
            local_combinations = [combinations[i][0], combinations[i][1]]

        for j in range(len(local_combinations)):
            if not F[k-1].get(local_combinations[j], False):
                combinations.pop(i)
                break

    print(f"amount of combinations after reduction: \t{len(combinations)}")
    print()

    # для каждой комбинации найдем поддержку
    for combination in combinations:
        col_mult = df[combination[0]].copy()
        for i in range(1, len(combination)):
            col_mult *= df[combination[i]]

        local_support = np.sum(col_mult) / df.shape[1]

        print(f"support of {combination}: \t{local_support}")

        # если поддержка больше установленной, то набор проходит на следующий этап
        if local_support >= support:
            C[combination] = np.sum(col_mult) / df.shape[1]

    F.append(C)

    print(f"frequent sets: {F[k]}")
    print()

    k = k + 1

Step 0
support of A: 	0.625
support of B: 	0.5
support of C: 	0.625
support of D: 	0.75
support of F: 	0.5
support of E: 	0.125
support of G: 	0.625
frequent sets: {'A': 0.625, 'B': 0.5, 'C': 0.625, 'D': 0.75, 'F': 0.5, 'G': 0.625}

Step 1
amount of possible combinations of 2 elements: 	21
amount of combinations after reduction: 	15

support of ('A', 'B'): 	0.375
support of ('A', 'C'): 	0.375
support of ('A', 'D'): 	0.5
support of ('A', 'F'): 	0.25
support of ('A', 'G'): 	0.25
support of ('B', 'C'): 	0.25
support of ('B', 'D'): 	0.25
support of ('B', 'F'): 	0.125
support of ('B', 'G'): 	0.25
support of ('C', 'D'): 	0.5
support of ('C', 'F'): 	0.25
support of ('C', 'G'): 	0.375
support of ('D', 'F'): 	0.5
support of ('D', 'G'): 	0.375
support of ('F', 'G'): 	0.25
frequent sets: {('A', 'B'): 0.375, ('A', 'C'): 0.375, ('A', 'D'): 0.5, ('C', 'D'): 0.5, ('C', 'G'): 0.375, ('D', 'F'): 0.5, ('D', 'G'): 0.375}

Step 2
amount of possible combinations of 3 elements: 	35
amount of combinations af

In [263]:
df = pd.DataFrame(zip(tid, itemset_list), columns=["tid", "itemset"])
df.head()

,tid,itemset
0,1,"[A, B, C, D]"
1,2,"[A, C, D, F]"
2,3,"[A, C, D, E, G]"
3,4,"[A, B, D, F]"
4,5,"[B, C, G]"


In [264]:
unique_items_count = dict()

for u_item in unique_items:
    count = 0
    for item in itemset:
        if u_item in item:
            count += 1
    unique_items_count[u_item] = count

In [268]:
df["itemset_ordered"] = [sorted(df["itemset"][i], key=lambda x : unique_items_count[x], reverse=True) for i in range(len(df["itemset"]))]
df.head(8)

,tid,itemset,itemset_ordered
0,1,"[A, B, C, D]","[D, A, C, B]"
1,2,"[A, C, D, F]","[D, A, C, F]"
2,3,"[A, C, D, E, G]","[D, A, C, G, E]"
3,4,"[A, B, D, F]","[D, A, B, F]"
4,5,"[B, C, G]","[C, G, B]"
5,6,"[D, F, G]","[D, G, F]"
6,7,"[A, B, G]","[A, G, B]"
7,8,"[C, D, F, G]","[D, C, G, F]"


In [266]:
root = Node("root", ind=0)

for i in range(len(df["itemset_ordered"])):
    prev = root
    for j in range(len(df["itemset_ordered"][i])):
        cur = None
        for child in prev.children:
            if df["itemset_ordered"][i][j] == child.name:
                cur = child

        if cur:
            cur.ind += 1
        else:
            cur = Node(df["itemset_ordered"][i][j], ind = 1, parent=prev)

        prev = cur


In [267]:
for pre, fill, node in RenderTree(root):
    print(f"{pre} {node.name} {node.ind}")


 root 0
├──  D 6
│   ├──  A 4
│   │   ├──  C 3
│   │   │   ├──  B 1
│   │   │   ├──  F 1
│   │   │   └──  G 1
│   │   │       └──  E 1
│   │   └──  B 1
│   │       └──  F 1
│   ├──  G 1
│   │   └──  F 1
│   └──  C 1
│       └──  G 1
│           └──  F 1
├──  C 1
│   └──  G 1
│       └──  B 1
└──  A 1
    └──  G 1
        └──  B 1
